In [29]:
import pandas as pd
import csv
from pytrends.request import TrendReq

In [30]:
pytrend = TrendReq()

In [31]:
with open('geoInfo.csv') as csv_file:
    reader = csv.reader(csv_file)
    geo_dict = dict(reader)

In [32]:
Americas = eval(geo_dict['Americas'])
EMEA = eval(geo_dict['EMEA'])
APAC = eval(geo_dict['APAC'])

In [ ]:
Americas

In [33]:
kw_list = ['Sony mirrorless', 'Canon mirrorless', 'Nikon mirrorless']
pytrend.build_payload(kw_list, timeframe='2020-01-01 2020-03-31')

In [ ]:
# df = pytrend.get_historical_interest(kw_list, year_start=2006, month_start=1, day_start=1, hour_start=0, year_end=2020, month_end=4, day_end=1, hour_end=0, cat=0, geo='CA,US,MX', gprop='', sleep=30)
df = pytrend.get_historical_interest(kw_list, cat=0, geo='', gprop='', sleep=30)
df.head(10)

In [34]:
# Interest by Region
df_region = pytrend.interest_by_region()

In [46]:
print(df_region.head(10))
print('..............................')
print(df_region.tail(10))
print('..............................')
print(df_region.shape)

Sony mirrorless  Canon mirrorless  Nikon mirrorless
geoName                                                               
Afghanistan                      0                 0                 0
Albania                          0                 0                 0
Algeria                          0                 0                 0
American Samoa                   0                 0                 0
Andorra                          0                 0                 0
Angola                           0                 0                 0
Anguilla                         0                 0                 0
Antarctica                       0                 0                 0
Antigua & Barbuda                0                 0                 0
Argentina                       43                38                19
..............................
                 Sony mirrorless  Canon mirrorless  Nikon mirrorless
geoName                                                            

In [47]:
kw_list = ['Sony mirrorless', 'Canon mirrorless', 'Nikon mirrorless']
pytrend.build_payload(kw_list, timeframe='2006-01-01 2006-03-31')
df_region = pytrend.interest_by_region()

In [48]:
print(df_region.head(10))
print('..............................')
print(df_region.tail(10))
print('..............................')
print(df_region.shape)

Sony mirrorless  Canon mirrorless  Nikon mirrorless
geoName                                                               
Afghanistan                      0                 0                 0
Albania                          0                 0                 0
Algeria                          0                 0                 0
American Samoa                   0                 0                 0
Andorra                          0                 0                 0
Angola                           0                 0                 0
Anguilla                         0                 0                 0
Antarctica                       0                 0                 0
Antigua & Barbuda                0                 0                 0
Argentina                        0                 0                 0
..............................
                 Sony mirrorless  Canon mirrorless  Nikon mirrorless
geoName                                                            

In [56]:
print(df_region.loc['United States','Canon mirrorless'])

0


In [ ]:
# United States, Japan



In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.tail()